# CHRTOUT access times

This notebook explore data access times for the CHRTOUT (channel routing output) Zarr store and for its subset (to gages known to the NWM) in CHANOBS Zarr store (generated in this 
[notebook](https://nbviewer.org/github/NCAR/rechunk_retro_nwm_v21/blob/main/notebooks/usage_example_rerechunk_chrtout.ipynb)). 

We look at retrieving:

    * A full timeseries from 1, 2, & 4 spatial point located in separate spatial chunks
    * A full spatial map from 1, 2, & 4 times located in separate time chunks (2 and 4 not applicable to CHANOBS since there is only 1 time chunk)
    
We do the above with and without dask.

In [1]:
import numpy as np
import pathlib
from pprint import pprint
import timeit
import xarray as xr

In [2]:
chrtout_file = '/glade/p/datashare/ishitas/nwm_retro_v2.1/chrtout.zarr'
dir_scratch = pathlib.Path('/glade/scratch/jamesmcc')
chanobs_file = str(dir_scratch / 'chanobs.zarr')

In [3]:
var = 'streamflow'
test_dict = {}


def pts_diff_chunks(n_chunks, chunk_size):
    return np.cumsum(np.arange(n_chunks) * chunk_size).tolist()


for ff in [chrtout_file, chanobs_file]:
    ds = xr.open_zarr(ff)

    full_time = slice(0, len(ds.time))
    full_feat = slice(0, len(ds.feature_id))
    
    time_chunk_size = ds[var].chunks[0][0]  # these are hard-coded to the order in chunks
    feat_chunk_size = ds[var].chunks[1][0]

    one_feat = pts_diff_chunks(1, feat_chunk_size) # pull them from differet chunks
    two_feat = pts_diff_chunks(2, feat_chunk_size)
    four_feat = pts_diff_chunks(4, feat_chunk_size)

    one_time = pts_diff_chunks(1, time_chunk_size) # pull them from differet chunks
    two_time = pts_diff_chunks(2, time_chunk_size)
    four_time = pts_diff_chunks(4, time_chunk_size)
    
    test_dict[ff] = {
        'full time, single feat': {'time': full_time, 'feature_id': one_feat},
        'full time, two feat': {'time': full_time, 'feature_id': two_feat},
        'full time, four feat': {'time': full_time, 'feature_id': four_feat},
        'single time, full feat': {'time': one_time, 'feature_id': full_feat},
    }
    if len(ds.time) / time_chunk_size < 1:
        test_dict['two time, full feat'] = {'time': two_time, 'feature_id': full_feat}
        test_dict['four time, full feat'] = {'time': four_time, 'feature_id': full_feat}
        
    ds.close()
    del ds
    


In [4]:
timing_results = {}
for ff, dd in test_dict.items():
    print(f'\n\n{ff}')
    timing_results[ff] = {
        'dask': {}, 
        'no_dask': {}, }
    for kk, vv in dd.items():
        print(f'\n{kk}')
        print(vv)



/glade/p/datashare/ishitas/nwm_retro_v2.1/chrtout.zarr

full time, single feat
{'time': slice(0, 367439, None), 'feature_id': [0]}

full time, two feat
{'time': slice(0, 367439, None), 'feature_id': [0, 30000]}

full time, four feat
{'time': slice(0, 367439, None), 'feature_id': [0, 30000, 90000, 180000]}

single time, full feat
{'time': [0], 'feature_id': slice(0, 2776738, None)}


/glade/scratch/jamesmcc/chanobs.zarr

full time, single feat
{'time': slice(0, 367439, None), 'feature_id': [0]}

full time, two feat
{'time': slice(0, 367439, None), 'feature_id': [0, 1]}

full time, four feat
{'time': slice(0, 367439, None), 'feature_id': [0, 1, 3, 6]}

single time, full feat
{'time': [0], 'feature_id': slice(0, 7994, None)}


In [5]:
# This is what we will be timing, open dataset, load data, close dataset
def data_load(case_name, file_zarr):
    arg_dict = test_dict[str(file_zarr)][case_name]
    ds = xr.open_zarr(file_zarr)
    da = ds.streamflow.isel(**arg_dict)
    display(da)
    da.load()
    ds.close()
    del ds
    del da
    return None


def run_timing_tests(file_zarr):
    print(f'Results for file: {file_zarr}\n')
    for test_key in test_dict[file_zarr].keys():
        print(f'\n{test_key}')
        time = timeit.timeit(
            stmt=f'data_load("{test_key}", "{file_zarr}")', 
            setup="from __main__ import data_load",
            number=n_trials)
        avg_time = time / n_trials
        timing_results[file_zarr][dask_status][test_key] = avg_time
        print(f'Avg time: {avg_time}')
    return None

## Without Dask

In [6]:
dask_status = 'no_dask'
n_trials = 1
for test_key in test_dict.keys():
    run_timing_tests(test_key)    

Results for file: /glade/p/datashare/ishitas/nwm_retro_v2.1/chrtout.zarr


full time, single feat


<xarray.DataArray 'streamflow' (time: 367439, feature_id: 1)>
dask.array<getitem, shape=(367439, 1), dtype=float64, chunksize=(672, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101
    gage_id     (feature_id) |S15 dask.array<chunksize=(1,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(1,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 124.62328261416405

full time, two feat


<xarray.DataArray 'streamflow' (time: 367439, feature_id: 2)>
dask.array<getitem, shape=(367439, 2), dtype=float64, chunksize=(672, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 434164
    gage_id     (feature_id) |S15 dask.array<chunksize=(2,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 289.9025640692562

full time, four feat


...

Avg time: 0.14610674139112234

full time, four feat


<xarray.DataArray 'streamflow' (time: 367439, feature_id: 4)>
dask.array<getitem, shape=(367439, 4), dtype=float64, chunksize=(367439, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(4,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 3923 13584 83230
    gage_id     (feature_id) |S15 dask.array<chunksize=(4,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(4,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 0.1679718093946576

single time, full feat


<xarray.DataArray 'streamflow' (time: 1, feature_id: 7994)>
dask.array<getitem, shape=(1, 7994), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 3923 12932 ... 1170023539 1180000535
    gage_id     (feature_id) |S15 dask.array<chunksize=(7994,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 119.23281815834343


### With Dask

In [7]:
import dask
from dask.distributed import Client, progress, LocalCluster, performance_report
from dask_jobqueue import PBSCluster

In [8]:
n_workers = 10
n_cores = 1
queue = "casper"
cluster_mem_gb = 15
chunk_mem_factor = 0.9

# numcodecs.blosc.use_threads = False

cluster = PBSCluster(
    cores=n_cores,
    memory=f"{cluster_mem_gb}GB",
    queue=queue,
    project="NRAL0017",
    walltime="05:00:00",
    death_timeout=75,)

cluster.adapt(maximum=n_workers, minimum=n_workers)
client = Client(cluster)
print(client.dashboard_link)

http://128.117.208.88:8787/status



- - - 
Pause here until the resuorces are allocated... still not sure how to do this programatically... 
- - - 

In [9]:
dask_status = 'dask'
n_trials = 1
for test_key in test_dict.keys():
    run_timing_tests(test_key)    

Results for file: /glade/p/datashare/ishitas/nwm_retro_v2.1/chrtout.zarr


full time, single feat


<xarray.DataArray 'streamflow' (time: 367439, feature_id: 1)>
dask.array<getitem, shape=(367439, 1), dtype=float64, chunksize=(672, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101
    gage_id     (feature_id) |S15 dask.array<chunksize=(1,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(1,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 64.36355334427208

full time, two feat


<xarray.DataArray 'streamflow' (time: 367439, feature_id: 2)>
dask.array<getitem, shape=(367439, 2), dtype=float64, chunksize=(672, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 434164
    gage_id     (feature_id) |S15 dask.array<chunksize=(2,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 36.581548262387514

full time, four feat


...

Avg time: 0.16793951485306025

full time, four feat


<xarray.DataArray 'streamflow' (time: 367439, feature_id: 4)>
dask.array<getitem, shape=(367439, 4), dtype=float64, chunksize=(367439, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(4,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 3923 13584 83230
    gage_id     (feature_id) |S15 dask.array<chunksize=(4,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(4,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 0.1763955894857645

single time, full feat


<xarray.DataArray 'streamflow' (time: 1, feature_id: 7994)>
dask.array<getitem, shape=(1, 7994), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 3923 12932 ... 1170023539 1180000535
    gage_id     (feature_id) |S15 dask.array<chunksize=(7994,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(7994,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(7994,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

Avg time: 29.04174958076328


In [13]:
import pandas as pd

In [15]:
pd.DataFrame.from_dict({
    (dd, cc): timing_results[dd][cc] 
    for dd in timing_results.keys() 
    for cc in timing_results[dd].keys()},
    orient='index')

full time, single feat  \
/glade/p/datashare/ishitas/nwm_retro_v2.1/chrto... dask                  64.363553   
                                                   no_dask              124.623283   
/glade/scratch/jamesmcc/chanobs.zarr               dask                   0.202661   
                                                   no_dask                0.216213   

                                                            full time, two feat  \
/glade/p/datashare/ishitas/nwm_retro_v2.1/chrto... dask               36.581548   
                                                   no_dask           289.902564   
/glade/scratch/jamesmcc/chanobs.zarr               dask                0.167940   
                                                   no_dask             0.146107   

                                                            full time, four feat  \
/glade/p/datashare/ishitas/nwm_retro_v2.1/chrto... dask                72.506712   
                                                   no_dask            665.464509   
/glade/scratch/jamesmcc/chanobs.zarr               dask                 0.176396   
                                                   no_dask              0.167972   

                                                            single time, full feat  
/glade/p/datashare/ishitas/nwm_retro_v2.1/chrto... dask                   4.566007  
                                                   no_dask               21.129391  
/glade/scratch/jamesmcc/chanobs.zarr               dask                  29.041750  
                                                   no_dask              119.232818